# TCGA RNA-seq hgr1 alignments Analysis I
```
pi:ababaian
files: ~/Crown/data2/tcga_analysis_1
start: 2018 09 14
complete : 2018 08 XX
```
## Introduction

hgr1 Alignment of the TCGA-XXX data. Only libraries for which there is a matched-normal from the same sample.


## Materials and Methods

#### TCGA Data - 1248 macp

Data checkpoint `180914-tcga-paired` / `180914_1248macp_vcf`


In [2]:
WORKDIR='/home/artem/Crown/data2/tcga_analysis'
cd $WORKDIR

In [3]:
# TCGA-Analysis Download data files
cd $WORKDIR

aws s3 cp --recursive s3://crownproject/tcga/180914_1248macp_vcf ./vcf/

download: s3://crownproject/tcga/180914_1248macp_vcf/18S_1248macp.vcf.log to vcf/18S_1248macp.vcf.log
download: s3://crownproject/tcga/180914_1248macp_vcf/18S_1248macp.vcf.bamlist to vcf/18S_1248macp.vcf.bamlist
download: s3://crownproject/tcga/180914_1248macp_vcf/TCGA-BLCA.18S_1248macp.vcf to vcf/TCGA-BLCA.18S_1248macp.vcf
download: s3://crownproject/tcga/180914_1248macp_vcf/TCGA-COAD.18S_1248macp.vcf to vcf/TCGA-COAD.18S_1248macp.vcf
download: s3://crownproject/tcga/180914_1248macp_vcf/TCGA-BRCA.18S_1248macp.vcf to vcf/TCGA-BRCA.18S_1248macp.vcf
download: s3://crownproject/tcga/180914_1248macp_vcf/TCGA-KICH.18S_1248macp.vcf to vcf/TCGA-KICH.18S_1248macp.vcf
download: s3://crownproject/tcga/180914_1248macp_vcf/TCGA-CESC.18S_1248macp.vcf to vcf/TCGA-CESC.18S_1248macp.vcf
download: s3://crownproject/tcga/180914_1248macp_vcf/TCGA-CHOL.18S_1248macp.vcf to vcf/TCGA-CHOL.18S_1248macp.vcf
download: s3://crownproject/tcga/180914_1248macp_vcf/TCGA-18S_1248.tsv to vcf/TCGA-18S_1248.tsv
download

#### TCGA Data - GVCF

In addition to the targetted analysis above; GVCF files for each TCGA cancer type were generated from the hgr1 alignments across 5S, 5.8S, 18S and 28S.

The `ADcalc.sh` script was modified (see below) to run this. Opened an M4.xlarge AWS Crown instance. The entire run through took ~2 days of run-time to process all cancers.

Zipped data stored on s3 at https://s3-us-west-2.amazonaws.com/crownproject/tcga/180926_tcga_gvcf/180926_tcga_gvcf.tar.gz . Downloaded to the workdir `gvcf` folder.

#### GVCF Script and Logs

...


In [3]:
cd $WORKDIR

cat gvcf/ADcalc.sh

echo '\n\n\n'

cat gvcf/ADcalc_gvcf.log

cat gvcf/ADcalc_gvcf_2.log

#!/bin/bash
# ADcalc.sh
# Allelic Depth Calculator
# for a position

cd ~/tcga/

# Controls -----------------
DEPTH='100000'
BAMLIST='bam.list.tmp'

#regions in hgr1.fa reference genome
REGIONS=('chr13:1003660-1005529' 'chr13:10219-10340' \
	'chr13:1006622-1006779' 'chr13:1007948-1013018')

#corresponding gene names
GENES=('18S' '5S' '5.8S' '28S')

# Iterate through every TCGA Cancer Type
for TYPE in $(ls)
do
    echo Analyzing $TYPE...
    cd $TYPE

    ls *.bam > bam.list.tmp
    ls *.bam >> ../$OUTPUT.bamlist
    ls *.bam >> ../tcga.bamlist

    for index in ${!GENES[*]}
    do
      printf "Started processing %s\n" ${GENES[$index]}
      OUTPUT="../$TYPE.${GENES[$index]}.gvcf"

      # Iterate through every bam file in directory
      # look-up position and return VCF
      bcftools mpileup -f ~/resources/hgr1/hgr1.fa \
        --max-depth $DEPTH -A --min-BQ 30 \
        -a FORMAT/DP,AD \
        -r ${REGIONS[$index]} \
        --ignore-RG \
        -b $BAMLIST | \
        bcftools

## Analysis Script

To analyze data; opened R-studio and used an R markdown file.

See: `./1248macp_quantification.Rmd`
